<a href="https://colab.research.google.com/github/MohammadNouman17/Digit-Recognition/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "First Order Motion Model for Image Animation"
To try the demo, press the 2 play buttons in order and scroll to the bottom. Note that it may take several minutes to load.

In [9]:
%%capture
# Install required dependencies
%pip install ffmpeg-python imageio-ffmpeg

# Initialize Git repository and clone the new model repository
!git init .
!git remote add origin https://github.com/graphemecluster/custom-first-order-model
!git pull origin main
!git clone https://github.com/graphemecluster/custom-first-order-model demo

# Clone new and diverse datasets

# 1. VoxCeleb2 dataset (video and audio for speaker recognition)
!git clone https://github.com/robots-oxford/voxceleb_voxceleb2.git /content/voxceleb2

# 2. UCF101 (human action recognition videos)
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar -O /content/UCF101.rar
!unrar x /content/UCF101.rar /content/UCF101

# 3. Open Images Dataset (large-scale diverse image dataset)
!git clone https://github.com/cvdfoundation/open-images-dataset.git /content/open-images

# 4. Kinetics-700 (videos for human activity understanding)
!wget --no-check-certificate https://storage.googleapis.com/deepmind-media/Datasets/kinetics700.tar.gz -O /content/kinetics700.tar.gz
!tar -xvzf /content/kinetics700.tar.gz -C /content/kinetics700

# 5. CelebV-HQ dataset (high-quality videos for face modeling)
!wget --no-check-certificate https://github.com/CelebV-HQ/celebv-hq/archive/refs/heads/main.zip -O /content/CelebV-HQ.zip
!unzip /content/CelebV-HQ.zip -d /content/celebv-hq

# 6. Sports-1M Dataset (sports-related video dataset)
!wget --no-check-certificate http://data.yt8m.org/download.py -O /content/sports-1m.py
!python3 /content/sports-1

In [12]:
# Update model list based on the links provided
model = ipywidgets.Dropdown(
    description="Model:",
    options=[
        'vox',
        'vox-adv',
        'taichi',
        'taichi-adv',
        'nemo',
        'mgif',
        'fashion',
        'bair'
    ]
)

# Custom behavior based on model selection
def change_model(change):
    if model.value.startswith('vox'):
        warning.remove_class('warn')  # Show or hide warning
    else:
        warning.add_class('warn')  # Show or hide warning
model.observe(change_model, names='value')

# Image upload logic
def upload_image(change):
    global selected_image
    for name, file_info in upload_input_image_button.value.items():
        content = file_info['content']
    if content is not None:
        selected_image = resize(PIL.Image.open(io.BytesIO(content)).convert("RGB"))
        input_image_widget.clear_output(wait=True)
        with input_image_widget:
            display(selected_image)
        input_image_widget.add_class('uploaded')
        display(Javascript('deselectImages()'))

upload_input_image_button.observe(upload_image, names='value')

# Video upload logic
def upload_video(change):
    global selected_video
    for name, file_info in upload_input_video_button.value.items():
        content = file_info['content']
    if content is not None:
        selected_video = 'user/' + name
        with open(selected_video, 'wb') as video:
            video.write(content)
        preview = resize(PIL.Image.fromarray(thumbnail(selected_video)).convert("RGB"))
        input_video_widget.clear_output(wait=True)
        with input_video_widget:
            display(preview)
        input_video_widget.add_class('uploaded')
        display(Javascript('deselectVideos()'))

upload_input_video_button.observe(upload_video, names='value')


def generate(button):
    main.layout.display = 'none'
    loading.layout.display = ''

    # Ensure correct checkpoint file is loaded based on the selected model
    filename = model.value + ('' if model.value == 'fashion' else '-cpk') + '.pth.tar'
    if not os.path.isfile(filename):
        response = requests.get('https://github.com/graphemecluster/first-order-model-demo/releases/download/checkpoints/' + filename, stream=True)
        with progress_bar:
            with tqdm.wrapattr(response.raw, 'read', total=int(response.headers.get('Content-Length', 0)), unit='B', unit_scale=True, unit_divisor=1024) as raw:
                with open(filename, 'wb') as file:
                    copyfileobj(raw, file)
        progress_bar.clear_output()

    reader = imageio.get_reader(selected_video, mode='I', format='FFMPEG')
    fps = reader.get_meta_data()['fps']
    driving_video = [frame for frame in reader]

    # Load the selected model checkpoint
    generator, kp_detector = load_checkpoints(config_path='config/%s-256.yaml' % model.value, checkpoint_path=filename)

    # Generate the animation with the selected video and image
    with progress_bar:
        predictions = make_animation(
            skimage.transform.resize(numpy.asarray(selected_image), (256, 256)),
            [skimage.transform.resize(frame, (256, 256)) for frame in driving_video],
            generator,
            kp_detector,
            relative=relative.value,
            adapt_movement_scale=adapt_movement_scale.value
        )
    progress_bar.clear_output()

    # Save the generated video as output
    imageio.mimsave('output.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

    # Combine the video with audio
    try:
        with NamedTemporaryFile(suffix='.mp4') as output:
            ffmpeg.output(ffmpeg.input('output.mp4').video, ffmpeg.input(selected_video).audio, output.name, c='copy').run()
            with open('output.mp4', 'wb') as result:
                copyfileobj(output, result)
    except ffmpeg.Error:
        pass

    # Display the output video
    output_widget.clear_output(True)
    with output_widget:
        video_widget = ipywidgets.Video.from_file('output.mp4', autoplay=False, loop=False)
        video_widget.add_class('video')
        display(video_widget)

    # Display comparison video (original vs generated)
    comparison_widget.clear_output(True)
    with comparison_widget:
        video_widget = ipywidgets.Video.from_file(selected_video, autoplay=False, loop=False, controls=False)
        video_widget.add_class('video')
        display(video_widget)

    # Synchronize the playback between original and generated videos
    display(Javascript("""
    setTimeout(function() {
        (function(left, right) {
            left.addEventListener("play", function() { right.play(); });
            left.addEventListener("pause", function() { right.pause(); });
            left.addEventListener("seeking", function() { right.currentTime = left.currentTime; });
            right.muted = true;
        })(document.getElementsByClassName("video-left")[0], document.getElementsByClassName("video-right")[0]);
    }, 1000);
    """))

    # Finalize UI update
    loading.layout.display = 'none'
    complete.layout.display = ''

def resize(image, size=(256, 256)):
    w, h = image.size
    d = min(w, h)
    r = ((w - d) // 2, (h - d) // 2, (w + d) // 2, (h + d) // 2)
    return image.resize(size, resample=PIL.Image.LANCZOS, box=r)

